# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

In [148]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

news = open('lenta.txt').read()
norm_news = normalize(news)

In [149]:
from collections import Counter

vocab_news = Counter(norm_news)
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})

In [150]:
from nltk.tokenize import sent_tokenize

def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

sentences_news = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)][:-100]
sentence_perp = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)][-100:]


In [151]:
unigrams_news = Counter()
bigrams_news = Counter()
threegrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    threegrams_news.update(ngrammer(sentence, n=3))

In [153]:
matrix_news = np.zeros((len(bigrams_news), 
                   len(unigrams_news)))

id2word_news_uni = list(unigrams_news)
word2id_news_uni = {word:i for i, word in enumerate(id2word_news_uni)}

id2word_news_bi = list(bigrams_news)
word2id_news_bi = {word:i for i, word in enumerate(id2word_news_bi)}

In [154]:
for ngram in threegrams_news:
    word1 = " ".join(ngram.split()[:2])
    word2 = ngram.split()[-1]
    matrix_news[word2id_news_bi[word1]][word2id_news_uni[word2]] =  (threegrams_news[ngram]/
                                                                     bigrams_news[word1])

In [155]:
def generate(matrix, id2word_bi, id2word_uni, word2id_bi, n=200, start='<start> <start>'):
    text = []
    current_idx = word2id_bi[start]
    for i in range(n):
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word_uni[chosen])
        if id2word_uni[chosen] == '<end>':
            current_idx = word2id_bi[start]
        else:
            part = id2word_bi[current_idx] + ' ' + id2word_uni[chosen]
            part = ' '.join(part.split()[1:])
            current_idx = word2id_bi[part]
    
    return ' '.join(text)

In [387]:
print(generate(matrix_news, id2word_news_bi, id2word_news_uni, word2id_news_bi).replace('<end>', '\n\n'))

гуманное изобретение сохранило тысячи жизней на сибирских этапах и пересылках 

 ранее предполагалось что именно наумов вместе с женой набирали код подъезда 

 по его оценке перспективы возможных переговоров с кредиторами о реструктуризации российских долговчастным инвесторам и кредиторам 

 этот ответ должен последовать рост рекламных расценок на каналах для обеспечения правопорядка а для назначения в администрацию президента путина накануне сегодняшнего заседания этот вопрос был ли это уголовное дело в том что некоторые документы для регистрации требуется указать серийный номер программы и интернет-приложения 

 c ледствие не исключает возможности расследования связанного с рассмотрением заявлений производится по плану предложенному бараком аната должна была пройти военная автоколонна и неподалеку от подмосковного солнечногорска столкнулись не менее представители microsoft уже опубликовала первую заплатку для internet explorer и скрипт-машиной которая непосредственно выполняет скрипт

In [176]:
def perplexity(probas):
    p = np.exp(np.sum(probas))
    N = len(probas)
    return p ** (-1/N) 

for sent in sentence_perp[:10]:
    prob = []
    for ngram in ngrammer(sent, n=3):
        word1 = " ".join(ngram.split()[:2])
        word2 = ngram.split()[-1]
        if ngram in threegrams_news and word1 in bigrams_news:
            prob.append(np.log(threegrams_news[ngram] / bigrams_news[word1]))
        else:
            prob.append(np.log(0.00001))
    print(" ".join(sent), perplexity(prob), '\n')

<start> <start> как сообщили ведомости глава кемеровской области аман тулеев уже выпустил постановление рекомендующее предприятиям региона рассмотреть возможность сотрудничества с глобальной русской информационной сетью grin ru <end> 21677.33893298278 

<start> <start> эта сеть была создана в начале года бывшим президентом московской фондовой биржи виктором сахаровым совместно с бывшим министром иностранных дел андреем козыревым и старается объединить все сектора экономики от сельского хозяйства до металлургии являясь таким образом многоотраслевым порталом b 2 b участниками которого стали уже 300 предприятий <end> 16496.45753273551 

<start> <start> в отличие от других видов электронной коммерции в 2 в привлекателен тем что денежные обороты здесь более значительны <end> 9101.96870185863 

<start> <start> так по данным агентства infoart news за 10 дней со дня открытия интернет-биржи зерно он-лайн где происходит торговля зерном сахаром подсолнечником и другими унифицированными сырьевыми 

## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

Во-первых, мы можем выбрать фиксированный словарь токенов, и ввести новый токен \<UNKN\>, который будем приписывать на этапе нормализации всем словам, которые не входят в словарь. Затем можем считать вероятность этого токена \<UNKN\> как любого другого

Во-вторых, мы можем создать "скрытый" словарь, приписывая токен \<UNKN\> всем словам, которые не проходят частотный порог, а затем снова обучить языковую модель, рассматривая \<UNKN\> как обычное слово

2. Что такое сглаживание (smoothing)?